In [21]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [22]:
import os
from pathlib import Path
import skimage.external.tifffile as tiff

from common import Statistics, dataset_source
from resources.conv_learner import *
from resources.plots import *

np.set_printoptions(precision=2)

In [23]:
PATH = "../datasets/yeast_v4"
data_path = Path(PATH)

In [24]:
CLASSES = ('WT', 'mfb1KO', 'mfb1KO_mmr1KO', 'mmr1KO')
NUM_CLASSES = len(CLASSES)
BATCH_SIZE = 64
SIZE = 200

In [25]:
test_dirs, train_dirs = dataset_source(data_path)
stats_tuple = Statistics.per_dataset(test_dirs, train_dirs)

8
working on a dataset with length: 714


In [33]:
def get_data(path: str, sz, bs, stats):
    create, lbl2index = ImageClassifierData.prepare_from_path(path, val_name='test', bs=bs, balance=True)
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomDihedral()], pad=sz//8) #even without transformations and padding -> failure
    print('\n class to index mapping:\n',lbl2index)
    return create(tfms)
balanced = get_data(PATH,SIZE, BATCH_SIZE,stats_tuple)


 class to index mapping:
 {'WT': 0, 'mfb1KO': 1, 'mfb1KO_mmr1KO': 2, 'mmr1KO': 3}
using <class 'torch.utils.data.sampler.WeightedRandomSampler'> sampler
using <class 'torch.utils.data.sampler.SequentialSampler'> sampler
using <class 'torch.utils.data.sampler.SequentialSampler'> sampler
using <class 'torch.utils.data.sampler.SequentialSampler'> sampler


In [34]:
ps = []
for _ in range(20): # simulation of 50 epochs
    for batch, y in balanced.trn_dl:
        ys = y.numpy().copy()
        counts = np.bincount(ys)
        percentages = [int(100* count / sum(counts)) for count in counts]
        ps.extend(percentages)
        
print(np.std(ps))

5.64951864653087


In [30]:
def get_data(path: str, sz, bs, stats):
    create, lbl2index = ImageClassifierData.prepare_from_path(path, val_name='test', bs=bs)
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomDihedral()], pad=sz//8) #even without transformations and padding -> failure
    print('\n class to index mapping:\n',lbl2index)
    return create(tfms)
unbalanced = get_data(PATH,SIZE, BATCH_SIZE,stats_tuple)


 class to index mapping:
 {'WT': 0, 'mfb1KO': 1, 'mfb1KO_mmr1KO': 2, 'mmr1KO': 3}


In [32]:
ps = []
for _ in range(20):
    for batch, y in unbalanced.trn_dl:
        ys = y.numpy().copy()
        counts = np.bincount(ys)
        percentages = [int(100* count / sum(counts)) for count in counts]
        ps.extend(percentages)
        
print(np.std(ps))

9.368647097500256
